# **MIP2_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

1.  X_{ij} : An array representing the number of products i produced in time period j.

2. Y_{ij} : An array representing the number of products i stored from time period j to time period j + 1.

3. P_{ij} : Array denoting fixed production cost of product i in time period j.

4. Q_{ij} : Array denoting variable production cost of product i in time period j.

5. S_{ij} : Array denoting variable storage cost of product i in time period j.

6. D_{ij} : An array representing demand of product i in each time period j.

7. R_{ij} : Array representing the revenue generated from selling product i in time period j.

8. production_capacity : An array representing the maximum number of each product that can be produced in each time period.

9. storage_capacity : An array representing the maximum number of each product that can be stored at the end of each time period.

That's it, these are required variables to formulate this optimization problem. Note that maximum demands are already represented

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

The objective function we need to maximize in this situation is the total profit, which is calculated as the total revenue minus the total cost. So, the equation to be maximized would be:

Maximize ∑_{j=1}^{12} ∑_{i=1}^{5} (R_{ij} * X_{ij} - P_{ij} - Q_{ij} * X_{ij} - S_{ij} * Y_{ij}) 

Where:
- i represents the product (1-2 for food and 3-5 for beverage)
- j represents the time period (1-12)
- X_{ij} represents the number of product i produced in time period j
- Y_{ij} represents the number of product i stored from time period j to j + 1
- R_{ij} represents the revenue generated from selling product i in period j
- P_{ij} represents the fixed production cost of product i in period j
- Q_{ij} represents the variable production cost of product i in period j
- S_{ij} represents the variable storage cost of product i from time period j to j + 1.


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

The constraints are as follows:

1. Production Capacity Constraints: 

For each product i and each time period j, the number of products produced should not exceed the production capacity:

X_{ij} ≤ Production Capacity[i] for all i in (1,5) and for all j in (1,12)

2. Storage Capacity Constraints:

For each product i and each time period j, the number of products stored should not exceed the storage capacity:

Y_{ij} ≤ Storage Capacity[i] for all i in (1,5) and for all j in (1,12)

3. Demand Constraints:

For each product i and each time period j, the number of products sold (produced and not stored) should not exceed the demand:

X_{ij} - Y_{ij} ≤ D_{ij} for all i in (1,5) and for all j in (1,12)

4. Continuity Constraints:

The number of products in stock at the end of period j should equal the number of products produced in period j plus the number of products left from period j-1 (stored from period j-1 to j):

Y_{ij} = X_{i,j-1} + Y_{i,j-1} for all i in (1,5) and for all j in (2,1

### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

First let's load necessary modules:

```python
from pyomo.environ import *
import numpy as np
```

Now we declare the model:

```python
model = ConcreteModel()
```

Next, we initialize sample data. In practice, this would come from the actual problem:

```python
N = 5 # number of products (5 in this case)
M = 12 # number of time periods (12 in this case)

np.random.seed(0)

# Sample data
P = np.random.rand(N,M) # fixed production cost
Q = np.random.rand(N,M) # variable production cost
S = np.random.rand(N,M) # variable storage cost
D = np.random.rand(N,M) # demand of product
R = np.random.rand(N,M) # revenue
production_capacity = np.random.rand(N,M) # production capacity
storage_capacity = np.random.rand(N,M) # storage capacity
```

Following, we define the decision variables:

```python
model.X = Var(range(N), range(M), within=NonNegativeReals) # products produced
model.Y = Var(range(N), range(M), within=NonNegativeReals) # products stored
```

Now, we define the objective:

```python
def objective_rule(model):
   return sum(R[i][j] * model.X[i,j] - P[i][j] - Q[i][j] * model.X[i,j] - S[i][j] * model.Y[i,j] for i in range(N) for j in range(M))
model.objective = Objective(rule=objective_rule, sense=maximize)
```

Moreover, we define the constraints:

```python
def production_capacity_rule(model, i, j):
   return model.X[i,j] <= production_capacity[i][j]
model.productions = Constraint(range(N), range(M), rule=production_capacity_rule)

def storage_capacity_rule(model, i, j):
   return model.Y[i,j] <= storage_capacity[i][j]
model.storage = Constraint(range(N), range(M), rule=storage_capacity_rule)

def demand_rule(model, i, j):
   return model.X[i, j] - model.Y[i, j] <= D[i][j]
model.demand = Constraint(range(N), range(M), rule=demand_rule)

def continuity_rule(model, i, j):
   if j == 0:
       return Constraint.Skip
   return model.Y[i, j] == model.X[i, j - 1] + model.Y[i, j - 1]
model.continuity = Constraint(range(N), range(M), rule=continuity_rule)
```

Finally, we run the model:

```python
solver = SolverFactory('gurobi') # define solver
solver.solve(model) # solve the model
```

Note: You can print the results and check for model status etc, I haven't included this here to only focus on building the optimization problem.

### **Run the code Generated by GPT4**

In [16]:
from pyomo.environ import *
import numpy as np

model = ConcreteModel()

N = 5 # number of products (5 in this case)
M = 12 # number of time periods (12 in this case)

np.random.seed(0)

# Sample data
P = np.random.rand(N,M) # fixed production cost
Q = np.random.rand(N,M) # variable production cost
S = np.random.rand(N,M) # variable storage cost
D = np.random.rand(N,M) # demand of product
R = np.random.rand(N,M) # revenue
production_capacity = np.random.rand(N,M) # production capacity
storage_capacity = np.random.rand(N,M) # storage capacity

model.X = Var(range(N), range(M), within=NonNegativeReals) # products produced
model.Y = Var(range(N), range(M), within=NonNegativeReals) # products stored

def objective_rule(model):
   return sum(R[i][j] * model.X[i,j] - P[i][j] - Q[i][j] * model.X[i,j] - S[i][j] * model.Y[i,j] for i in range(N) for j in range(M))
model.objective = Objective(rule=objective_rule, sense=maximize)

def production_capacity_rule(model, i, j):
   return model.X[i,j] <= production_capacity[i][j]
model.productions = Constraint(range(N), range(M), rule=production_capacity_rule)

def storage_capacity_rule(model, i, j):
   return model.Y[i,j] <= storage_capacity[i][j]
model.storage = Constraint(range(N), range(M), rule=storage_capacity_rule)

def demand_rule(model, i, j):
   return model.X[i, j] - model.Y[i, j] <= D[i][j]
model.demand = Constraint(range(N), range(M), rule=demand_rule)

def continuity_rule(model, i, j):
   if j == 0:
       return Constraint.Skip
   return model.Y[i, j] == model.X[i, j - 1] + model.Y[i, j - 1]
model.continuity = Constraint(range(N), range(M), rule=continuity_rule)

solver = SolverFactory('glpk') # define solver
solver.solve(model) # solve the model

{'Problem': [{'Name': 'unknown', 'Lower bound': -30.029318897895, 'Upper bound': -30.029318897895, 'Number of objectives': 1, 'Number of constraints': 235, 'Number of variables': 121, 'Number of nonzeros': 405, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.00816488265991211}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [37]:
from pyomo.environ import *
import numpy as np
import pandas as pd
model = ConcreteModel()

N = 5 # number of products (5 in this case)
M = 12 # number of time periods (12 in this case)

#data inputted by human

P = pd.read_csv("fixed_cost_production.csv")
P.index += 1
P = P.drop("Unnamed: 0", axis = 1)
P.columns = P.columns.astype(int)
P = P.to_numpy()


Q = pd.read_csv("variable_cost_production.csv")
Q.index += 1
Q = Q.drop("Unnamed: 0", axis = 1)
Q.columns = Q.columns.astype(int)
Q = Q.to_numpy()


S = pd.read_csv("variable_cost_storage.csv")
S.index += 1
S = S.drop("Unnamed: 0", axis = 1)
S.columns = S.columns.astype(int)
S = S.to_numpy()


D = pd.read_csv("demand.csv")
D.index += 1
D = D.drop("Unnamed: 0", axis = 1)
D.columns = D.columns.astype(int)
D = D.to_numpy()


R = pd.read_csv("revenue.csv")
R.index += 1
R = R.drop("Unnamed: 0", axis = 1)
R.columns = R.columns.astype(int)
R = R.to_numpy()

storage_capacity = {1: 580,
    2: 687,
    3: 599,
    4: 788,
    5: 294
}

production_capacity = {1: 1080,
    2: 908,
    3: 408,
    4: 1000,
    5: 403
}
#end

model.X = Var(range(N), range(M), within=NonNegativeReals) # products produced
model.Y = Var(range(N), range(M), within=NonNegativeReals) # products stored

def objective_rule(model):
   return sum(R[i][j] * model.X[i,j] - P[i][j] - Q[i][j] * model.X[i,j] - S[i][j] * model.Y[i,j] for i in range(N) for j in range(M))
model.objective = Objective(rule=objective_rule, sense=maximize)

def production_capacity_rule(model, i, j):
   return model.X[i,j] <= production_capacity[i+1]
model.productions = Constraint(range(N), range(M),rule=production_capacity_rule)

def storage_capacity_rule(model, i, j):
   return model.Y[i,j] <= storage_capacity[i+1]
model.storage = Constraint(range(N), range(M), rule=storage_capacity_rule)

def demand_rule(model, i, j):
   return model.X[i, j] - model.Y[i, j] <= D[i][j]
model.demand = Constraint(range(N), range(M), rule=demand_rule)

def continuity_rule(model, i, j):
   if j == 0:
       return Constraint.Skip
   return model.Y[i, j] == model.X[i, j - 1] + model.Y[i, j - 1]
model.continuity = Constraint(range(N), range(M), rule=continuity_rule)

solver = SolverFactory('glpk') # define solver
solver.solve(model) # solve the model
print(model.objective())

60876.915182421886


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**